In [1]:
import os
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [2]:
# LLM Dataset
data = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_essays.csv')
data = data[["text", "generated"]].copy()

In [3]:
temp1 = pd.read_csv('/kaggle/input/daigt-external-train-dataset/train_external_drcat_02.csv')
temp1 = temp1[["text", "label"]].copy()
temp1.rename(columns={"label": "generated"}, inplace=True)
temp1.dropna(inplace=True)

/tmp/ipykernel_19/3124174556.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  temp1 = pd.read_csv('/kaggle/input/daigt-external-train-dataset/train_external_drcat_02.csv')


In [4]:
t1 = pd.read_csv('/kaggle/input/daigt-proper-train-dataset/train_drcat_01.csv')
t2 = pd.read_csv('/kaggle/input/daigt-proper-train-dataset/train_drcat_02.csv')
t3 = pd.read_csv('/kaggle/input/daigt-proper-train-dataset/train_drcat_03.csv')
t4 = pd.read_csv('/kaggle/input/daigt-proper-train-dataset/train_drcat_04.csv')

temp2 = pd.concat([t1, t2, t3, t4], ignore_index=True)
temp2 = temp2[["text", "label"]].copy()
temp2.rename(columns={"label": "generated"}, inplace=True)
temp2.dropna(inplace=True)

In [5]:
t1 = pd.read_csv('/kaggle/input/daigt-v3-train-dataset/train_v3_drcat_01.csv')
t2 = pd.read_csv('/kaggle/input/daigt-v3-train-dataset/train_v3_drcat_02.csv')

temp3 = pd.concat([t1, t2], ignore_index=True)
temp3.rename(columns={"label": "generated"}, inplace=True)
temp3.dropna(inplace=True)

In [6]:
data = pd.concat([data, temp1, temp2, temp3], ignore_index=True)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(data["text"], data["generated"], test_size=0.2)

In [8]:
vectorizer = TfidfVectorizer(max_features=100000)
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

In [9]:
model = LogisticRegression(max_iter=1000000)
model.fit(X_train_vectorized, y_train)

LogisticRegression(max_iter=1000000)

In [10]:
accuracy = model.score(X_test_vectorized, y_test)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9926235659912083


In [11]:
input_data = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')
input_data = input_data[["id", "text"]].copy()

In [12]:
model_input = vectorizer.transform(input_data["text"])
pred = model.predict(model_input)

In [13]:
output_df = pd.DataFrame({
    "id": input_data["id"],
    "generated": pred
})
output_df.to_csv('/kaggle/working/submission.csv', index=False)